# Create and Destroy Device

__Device creation, vector operation and torch interop, destroying device__

Vulky is a python facade to vulkan with reduced and simpler interface focused primarly to academic purposes. Objects are designed to represent graphics pipelines and techniques in a more compact way.

In [ ]:
import torch

try: # install all dependencies in colab 
    import google.colab
    !sudo apt-get update -y
    !sudo apt-get install -y libnvidia-gl-555 vulkan-tools glslang-tools vulkan-validationlayers-dev
    !pip install pyav==13.1.0
    !pip install git+https://github.com/rendervous/vulky_project.git
except:
    print("Executing locally")

Let's start importing the module ```vulky```


In [ ]:
import vulky as vk

## Vectors and Matrices

Apart from rendering, **Vulky** provides object types for working with vectors and matrices, internally wrapping PyTorch tensors. This makes it easier to interpret tensors as 3D geometric entities while preserving differentiability whenever possible. These types also serve as representations of equivalent Vulkan shader types (e.g., `vec2`, ..., `vec4`; `mat2`, ..., `mat4x3`, `mat4`).

A key distinction is that, in **Vulky**, matrices are **row-major**, matching the memory layout of PyTorch tensors. Additionally, all transformations assume **row vectors**, which aligns with the typical shape of a batch of vectors in PyTorch (`B × D`).

In [ ]:
import torch
x = vk.vec3(1., 0., .0)
M = vk.mat3.rotation(vk.vec3(0., 0.0, 1.0), 3.14159/2)
y = x @ M  # proper way to transform a vertex in vulky
print(x)
print(y)

neg_y = M @ x  # also works because a special treatment of 1D tensors in PyTorch as column vectors. But has the transposed effect.
print(neg_y)

Vector and matrix types in **Vulky** also support batching. For instance, if you want to refer to a bidirectional array of shape `(16, 16)` containing `vec4` elements, you can use:

In [ ]:
t = vk.vec4.zero(16, 16)
print(t)
print(t.shape)

Operations involving multiple vectors and matrices will automatically **broadcast** to match the largest batch shape, following PyTorch-style broadcasting rules.

In [ ]:
t += vk.vec4(1.0, 0.2, 0.3, 0.4)
print(t)
print(t.shape)

An important observation is that indexing into vectors and matrices refers to their **components**, not to individual **batch instances**.  
For example:

In [ ]:
t[0] = 1.0  # first component of all (16,16) vec4 where set to 1.0
t.y = 2.0  # equivalent to index, a named access to the field is also valid and refers to whole the batch
print(t)  

## Devices ##

For graphics, **Vulky** internally operates with a single Vulkan instance at a time but can manage multiple devices. The concept of an *active device* is central: most methods in the Vulky library implicitly refer to this active device.

You can set the active device using the `device_manager` method by passing the desired device object. By default, the device created most recently becomes the active one.

If you don't plan to switch between devices later, it's not necessary to store a reference to the device — Vulky will continue to use the active one by default.

In [ ]:
vk.create_device(debug=True)

Vulky automatically manages two types of memory: memory accessible by the host (CPU) and memory local to the graphics device (GPU). 

When CUDA is available, the device memory is exported to CUDA and the PyTorch library, allowing for seamless creation of tensors that are backed by Vulkan-managed memory. This simplifies interoperability between Vulkan and PyTorch.

In [ ]:
t = vk.tensor(2,4)
print(t + 0.2)

Notice that those tensors are created in gpu memory. 

In [ ]:
print(t.device)

Also, for the vector types the library provides different random generators based on torch.

In [ ]:
a = vk.vec3.rand()  # U[0..1)
b = vk.vec3.randn()  # N(0, I)
c = vk.vec3.randd(1000)  # Uniform in hypersphere

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(3,3), dpi=200)
ax = fig.add_subplot(projection='3d')
ax.scatter(c.x, c.y, c.z)
ax.axis('off')
ax.set_box_aspect((1,1,1))
fig.tight_layout(pad=0.0)
fig.savefig('teaser1.jpg')
plt.show()

Although **Vulky** tensors can be used like regular `torch` tensors, the memory is owned and managed by Vulky. As a result, these tensors **must be explicitly deleted** before the associated Vulkan device is destroyed to avoid memory issues or crashes.


In [ ]:
del t
vk.quit()

In the rest of the notebooks we won't close explicitly the device, although it is automatically performed at exit, it is unpleasant when we want to repeat cell executions and the device is already destroyed.